<a href="https://colab.research.google.com/github/YoushaAdib/Python-Portfolio/blob/main/Bank_Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Bank Job
# Bank Reconciliation and Self-Audit


In [2]:
# Environment Setup 
from google.colab import drive 
drive.mount('/content/drive')

import pandas as pd
df_ledger = pd.read_csv ('/content/drive/MyDrive/1_UK/1_Yousha Adib/6_Portfolio Projects/Audiit & Accounts/Bank Job /bank_ledger.csv')
df_statement = pd.read_csv ('/content/drive/MyDrive/1_UK/1_Yousha Adib/6_Portfolio Projects/Audiit & Accounts/Bank Job /bank_statement.csv')

print(df_ledger.head())
print(df_statement.head())

print(df_ledger.info())
print(df_statement.info())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     TXN_DATE   REFERENCE    TXN_AMT
0  02/08/2022  PAY 007907     500.00
1  02/08/2022  REC 001095    1910.18
2  03/08/2022  REC 001102  250000.00
3  04/08/2022  PAY 007933    1050.00
4  04/08/2022  PAY 008142       8.50
    Bank_Date   Bank_Amt                                           Memo
0  11/05/2023   42000.00  M-R SOLICITORS LLP    \tCOMPLETION 11TH FL BB
1  10/05/2023  150000.00          202620 83764540       \tTRANSFER FT\t
2  09/05/2023       8.50  CHARGES               COMMISSION FOR\tPERIOD 
3  27/04/2023     217.79       M K JUGON             \tTRAVELLING BBP\t
4  27/04/2023     127.80       MOHAMMAD TOUSEEF      \tTRAVELLING BBP\t
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TXN_DATE   79 non-null     obj

In [4]:
# Date Conversion
df_ledger['TXN_DATE'] = pd.to_datetime(df_ledger['TXN_DATE'])
df_statement['Bank_Date'] = pd.to_datetime(df_statement['Bank_Date'])

# Outer Merge
df_recon = pd.merge(df_statement, df_ledger, how='outer', left_on=['Bank_Date', 'Bank_Amt'], right_on=['TXN_DATE', 'TXN_AMT'], indicator=True)
df_recon['Match'] = df_recon.apply(lambda row: 'Matched' if row['_merge'] == 'both' else ('Not Matched' if pd.notnull(row['Bank_Date']) else 'Not Found'), axis=1)
df_recon['Matched_Reference'] = df_recon.apply(lambda row: row['REFERENCE'] if row['Match'] == 'Matched' else ('Not Matched' if pd.notnull(row['Bank_Date']) else 'Not Found'), axis=1)

# Dropping Extra Column
df_recon = df_recon.drop(columns=['REFERENCE','_merge','Memo'])

df_recon.head(100)


,Bank_Date,Bank_Amt,TXN_DATE,TXN_AMT,Match,Matched_Reference
0,2023-11-05,42000.00,NaT,NaN,Not Matched,Not Matched
1,2023-10-05,150000.00,NaT,NaN,Not Matched,Not Matched
2,2023-09-05,8.50,NaT,NaN,Not Matched,Not Matched
3,2023-04-27,217.79,2023-04-27,217.79,Matched,PAY 008714
4,2023-04-27,127.80,2023-04-27,127.80,Matched,PAY 008715
...,...,...,...,...,...,...
82,2022-02-08,1910.18,2022-02-08,1910.18,Matched,REC 001095
83,2022-02-08,500.00,2022-02-08,500.00,Matched,PAY 007907
84,NaT,NaN,2022-06-09,250.00,Not Found,Not Found
85,NaT,NaN,2022-06-09,8.50,Not Found,Not Found


In [ ]:
# Printing Output 
df_recon.to_csv('/content/drive/MyDrive/1_UK/1_Yousha Adib/6_Portfolio Projects/Audiit & Accounts/Bank Job /df_recon.csv', index=False)